# Part 2 - Mapping Yelp Search Results

## Obective

- For this CodeAlong, we will be working with the Yelp API results from last class. 
- You will load in the .csv.gz of your yelp results and prepare the data for visualization.
- You will use Plotly Express to create an interactive map with all of the results.

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- [Advanced Transformations with Pandas - Part 1](https://login.codingdojo.com/m/376/12529/88086)
- [Advanced Transformations with Pandas - Part 2](https://login.codingdojo.com/m/376/12529/88088)

### Goal

- We want to create a map with every restaurant plotted as a scatter plot with detailed information that appears when we hover over a business
- We will use plotly express's `px.scatter_mapbox` function to accomplish this.
    - https://plotly.com/python/scattermapbox/
    
    - We will need a Mapbox API token for some of the options:
        - https://studio.mapbox.com/
    

# Loading Data from Part 1

In [1]:
## Plotly is not included in your dojo-env
!pip install plotly

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json

## importing plotly 
import plotly.express as px

In [3]:
## Load in csv.gz
df = pd.read_csv('Data/Reno-Sushi.csv.gz')
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,pms70Lg_z7c2y5ojukohBQ,miyabi-japanese-cuisine-reno,MIYABI Japanese Cuisine,https://s3-media1.fl.yelpcdn.com/bphoto/hE5VjD...,False,https://www.yelp.com/biz/miyabi-japanese-cuisi...,144,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 39.4035814750889, 'longitude': -1...",[],$$,"{'address1': '13967 S Virginia St', 'address2'...",1.775409e+10,(775) 409-3520,18553.049571
1,W9y4JkMqy2dFNwYc3Y5cKQ,hinoki-sushi-reno,Hinoki Sushi,https://s3-media3.fl.yelpcdn.com/bphoto/vwh7Ea...,False,https://www.yelp.com/biz/hinoki-sushi-reno?adj...,843,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 39.473410597855896, 'longitude': ...",['delivery'],$$,"{'address1': '5270 Longley Ln', 'address2': ''...",1.775409e+10,(775) 409-4994,11412.584176
2,3FPi2yKCIh0Hh3iBg8faYA,tokyo-sushi-reno-2,Tokyo Sushi,https://s3-media4.fl.yelpcdn.com/bphoto/WVhsaZ...,False,https://www.yelp.com/biz/tokyo-sushi-reno-2?ad...,978,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 39.50491, 'longitude': -119.80335}",[],$$,"{'address1': '1999 S Virginia St', 'address2':...",1.775826e+10,(775) 825-8828,6560.327423
3,gExn3d0VMMJWlSICsGHGfg,ohana-sushi-sparks-3,Ohana Sushi,https://s3-media2.fl.yelpcdn.com/bphoto/nMfIDn...,False,https://www.yelp.com/biz/ohana-sushi-sparks-3?...,495,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 39.52066, 'longitude': -119.740751}",[],$$,"{'address1': '1560 S Stanford Way', 'address2'...",1.775454e+10,(775) 453-9722,10134.379359
4,mc-U-3T6UMN9x7SVm9Yn3A,hinoki-o-reno-2,Hinoki O,https://s3-media4.fl.yelpcdn.com/bphoto/ZsNOKx...,False,https://www.yelp.com/biz/hinoki-o-reno-2?adjus...,26,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",5.0,"{'latitude': 39.46823189149659, 'longitude': -...",[],NaN,"{'address1': '7500 Rancharrah Pkwy', 'address2...",1.775500e+10,(775) 499-5818,10421.916314


## Required Preprocessing 

- 1. We need to get the latitude and longitude for each business as separate columns.
- We also want to be able to show the restaurants:
    - name,
    - price range
    - address
    - and if they do delivery or takeout.

### Separating Latitude and Longitude

In [4]:
## use .apply pd.Series to convert a dict to columns
df['coordinates'].apply(pd.Series)

,0
0,"{'latitude': 39.4035814750889, 'longitude': -1..."
1,"{'latitude': 39.473410597855896, 'longitude': ..."
2,"{'latitude': 39.50491, 'longitude': -119.80335}"
3,"{'latitude': 39.52066, 'longitude': -119.740751}"
4,"{'latitude': 39.46823189149659, 'longitude': -..."
...,...
255,"{'latitude': 38.917536, 'longitude': -120.011056}"
256,"{'latitude': 39.53139031542699, 'longitude': -..."
257,"{'latitude': 39.39505095, 'longitude': -119.76..."
258,"{'latitude': 39.55529322, 'longitude': -119.75..."


- Why didn't that work???

In [5]:
## slice out a single test coordinate
test_coord = df.loc[1, 'coordinates']
test_coord

"{'latitude': 39.473410597855896, 'longitude': -119.76392370361854}"

- Its not a dictionary anymore!!! WTF??
    - CSV files cant store iterables (lists, dictionaries) so they get converted to strings.

### Fixing the String-Dictionaries

- The json module has another version of load and dump called `json.loads` and `json.dumps`
    - These are designed to process STRINGS instead of files. 
    
- If we use `json.loads` we can convert our string dictionary into an actual dictionary. 

In [6]:
## Use json.loads on the test coordinate
json.loads(test_coord)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

- JSON requires double quotes!

In [7]:
## replace single ' with " 
test_coord = test_coord.replace("'", '"')
test_coord

'{"latitude": 39.473410597855896, "longitude": -119.76392370361854}'

In [8]:
## Use json.loads on the test coordinate, again
json.loads(test_coord)

{'latitude': 39.473410597855896, 'longitude': -119.76392370361854}

### Now, how can we apply this same process to the entire column??

In [9]:
## replace ' with " (entire column)
df['coordinates'] = df['coordinates'].str.replace("'", '"')
## apply json.loads
df['coordinates'] = df['coordinates'].apply(json.loads)

In [10]:
## slice out a single test coordinate
test_coord = df.loc[5, 'coordinates']
test_coord

{'latitude': 39.4841, 'longitude': -119.74679}

### Using Apply with pd.Series to convert a dictionary column into multiple columns

In [11]:
## use .apply pd.Series to convert a dict to columns
df['coordinates'].apply(pd.Series)

,latitude,longitude
0,39.403581,-119.748760
1,39.473411,-119.763924
2,39.504910,-119.803350
3,39.520660,-119.740751
4,39.468232,-119.793039
...,...,...
255,38.917536,-120.011056
256,39.531390,-119.832515
257,39.395051,-119.765624
258,39.555293,-119.754152


In [12]:
## Concatenate the 2 new columns and drop the original.
df = pd.concat([df, df['coordinates'].apply(pd.Series)], axis=1)
df = df.drop(columns='coordinates')
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,price,location,phone,display_phone,distance,latitude,longitude
0,pms70Lg_z7c2y5ojukohBQ,miyabi-japanese-cuisine-reno,MIYABI Japanese Cuisine,https://s3-media1.fl.yelpcdn.com/bphoto/hE5VjD...,False,https://www.yelp.com/biz/miyabi-japanese-cuisi...,144,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,[],$$,"{'address1': '13967 S Virginia St', 'address2'...",1.775409e+10,(775) 409-3520,18553.049571,39.403581,-119.748760
1,W9y4JkMqy2dFNwYc3Y5cKQ,hinoki-sushi-reno,Hinoki Sushi,https://s3-media3.fl.yelpcdn.com/bphoto/vwh7Ea...,False,https://www.yelp.com/biz/hinoki-sushi-reno?adj...,843,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,['delivery'],$$,"{'address1': '5270 Longley Ln', 'address2': ''...",1.775409e+10,(775) 409-4994,11412.584176,39.473411,-119.763924
2,3FPi2yKCIh0Hh3iBg8faYA,tokyo-sushi-reno-2,Tokyo Sushi,https://s3-media4.fl.yelpcdn.com/bphoto/WVhsaZ...,False,https://www.yelp.com/biz/tokyo-sushi-reno-2?ad...,978,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,[],$$,"{'address1': '1999 S Virginia St', 'address2':...",1.775826e+10,(775) 825-8828,6560.327423,39.504910,-119.803350
3,gExn3d0VMMJWlSICsGHGfg,ohana-sushi-sparks-3,Ohana Sushi,https://s3-media2.fl.yelpcdn.com/bphoto/nMfIDn...,False,https://www.yelp.com/biz/ohana-sushi-sparks-3?...,495,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,[],$$,"{'address1': '1560 S Stanford Way', 'address2'...",1.775454e+10,(775) 453-9722,10134.379359,39.520660,-119.740751
4,mc-U-3T6UMN9x7SVm9Yn3A,hinoki-o-reno-2,Hinoki O,https://s3-media4.fl.yelpcdn.com/bphoto/ZsNOKx...,False,https://www.yelp.com/biz/hinoki-o-reno-2?adjus...,26,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",5.0,[],NaN,"{'address1': '7500 Rancharrah Pkwy', 'address2...",1.775500e+10,(775) 499-5818,10421.916314,39.468232,-119.793039


## Creating a Simple Map

### Register for MapBox API

Mapbox API: https://www.mapbox.com/

In [13]:
## Load in mapbox api credentials from .secret
with open('/Users/joshl/.secret/mapbox.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

- Use the plotly express `set_maptbox_acccess_token` function

In [ ]:
## set mapbox token


In [ ]:
## use scatter_mapbox for M.V.P map


### Adding Hover Data

- We want to show the restaurants:
    - name
    - price range
    - address
    - and if they do delivery or takeout.
    
    
- We can use the `hover_name` and `hover_data` arguments for `px.scatter_mapbox` to add this info!

In [ ]:
## add hover_name (name) and hover_data for price,rating,location


### Fixing the Location Column

In [ ]:
## slice out a test address


> Also a string-dictionary...

In [ ]:
## replace ' with "
df['location'] = df['location'].str.replace("'", '"')
df

In [ ]:
## apply json.loads


> Ruh roh....

- Hmm, let's slice out a test_address again and let's write a function to accomplish this instead.
    - We can use try and except in our function to get around the errors.

### Fixing Addresses - with a custom function


In [ ]:
## slice out test address 
test_addr = df.loc[0, 'location']
test_addr

In [ ]:
## write a function to just run json.loads on the address


In [ ]:
## test applying our function


- It worked! Now let's save this as a new column (display_location),
and then let's investigate the businesses that had an "ERROR".

In [ ]:
### save a new display_location column using our function


In [ ]:
## filter for businesses with display_location == "ERROR"


In [ ]:
## slice out a new test address and inspect
test_addr = df.loc[437, 'location']
test_addr

> After some more investigation, we would find a few issues with these "ERROR" rows.
1. They contained None.
2. They contained an apostrophe in the name.
3. ...?

### Possible Fixes (if we care to/have the time)


- Use Regular Expressions to find an fix the display addresses with "'" in them
- Use string split to split on the word display address.
    - Then use string methods to clean up

### Moving Forward without those rows (for now)

In [ ]:
## remove any rows where display_location == 'ERROR'


- We want the "display_address" key from the "display_location" dictionaries.
- We could use a .apply and a lamda to slice out the desired key.

In [ ]:
## use apply and lambda to slice correct key


- Almost done! We want to convert display_address to a string instead a list of strings.
- We can use the string method .join to do so!

In [ ]:
## slice out a test_address


In [ ]:
## test using .join with a "\n"


In [ ]:
## apply the join to every row with a lambda


### Final Map

In [ ]:
## make ourn final map and save as varaible


#### HTML Uses `<br>` instead of `\n`

In [ ]:
## remake the final address column with <br> instead 

## plot the final map

In [ ]:
## use fig.write_html to save map
